In [4]:
import torch
from torch import nn

def change_maskrcnn_benchmark_model(model_path: str, trimmed_model_path: str):
    state_dict = torch.load(model_path, map_location="cpu")
    
    model = state_dict['model']
    cls_weight_new = torch.Tensor(271+320*3, 1024)
    nn.init.normal_(cls_weight_new, std=0.01)
    cls_weight_new[:271+320*2] = model['module.roi_heads.box.predictor.cls_score.weight']
    model['module.roi_heads.box.predictor.cls_score.weight'] = cls_weight_new
    
    mask_weight_new = torch.Tensor(271+320*3, 256, 1, 1)
    nn.init.normal_(mask_weight_new, std=0.01)
    mask_weight_new[:271+320*2] = model['module.roi_heads.mask.predictor.mask_fcn_logits.weight']
    model['module.roi_heads.mask.predictor.mask_fcn_logits.weight'] = mask_weight_new
    
    mask_bias_new = torch.Tensor(271+320*3)
    nn.init.normal_(mask_bias_new, std=0.01)
    mask_bias_new[:271+320*2] = model['module.roi_heads.mask.predictor.mask_fcn_logits.bias']
    model['module.roi_heads.mask.predictor.mask_fcn_logits.bias'] = mask_bias_new
    
    
    reg_weight_new = torch.Tensor((271+320*3)*4, 1024)
    nn.init.normal_(reg_weight_new, std=0.001)
    reg_weight_new[:(271+320*2)*4] = model['module.roi_heads.box.predictor.bbox_pred.weight']
    model['module.roi_heads.box.predictor.bbox_pred.weight'] = reg_weight_new
    
    reg_bias_new = torch.Tensor((271+320*3)*4)
    nn.init.constant_(reg_bias_new, 0)
    reg_bias_new[:(271+320*2)*4] = model['module.roi_heads.box.predictor.bbox_pred.bias']
    model['module.roi_heads.box.predictor.bbox_pred.bias'] = reg_bias_new
    
#     import pdb; pdb.set_trace()
#     for key in keys_to_change:
#         if key in model:

#             weight_ex = torch.Tensor(141+40+40+40, 1024)
#             nn.init.normal_(weight_ex, std=0.01)
#             weight_ex[:141+40+40] = model[key][:141+40+40]
# #             weight_ex= torch.tensor(step2_cls_weights)
#             model[key] = weight_ex
#             print('key: {} is changed'.format(key))
#         else:
#             print('key: {} is not present'.format(key))

    print("Also deleting optimizer, scheduler, and iteration entries")
    del state_dict['optimizer']
    del state_dict['scheduler']
    del state_dict['iteration']
    
    torch.save(state_dict, trimmed_model_path)
    print(f'saved to: {trimmed_model_path}')

In [5]:
model_path = "/gruntdata0/xinting/project/LVIS_new/maskxrcnn_finetune/ckps/ckp-step2_freeze4_wo_distill_balance_320/model_0008000.pth"
state_dict = torch.load(model_path, map_location="cpu")
model = state_dict['model']
trimmed_model_path = "/gruntdata0/xinting/project/LVIS_new/maskxrcnn_finetune/ckps/ckp-step2_freeze4_wo_distill_balance_320/model_0008000_for_finetune.pth"
# /mnt/data-disk0/xinting/LVIS/maskrcnn-benchmark/ckp-fewshot-novel

In [6]:
change_maskrcnn_benchmark_model(model_path, trimmed_model_path)

Also deleting optimizer, scheduler, and iteration entries
saved to: /gruntdata0/xinting/project/LVIS_new/maskxrcnn_finetune/ckps/ckp-step2_freeze4_wo_distill_balance_320/model_0008000_for_finetune.pth
